In [ ]:
pip install langchain openai flask requests beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import requests
from bs4 import BeautifulSoup

def extract_courses(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    courses = []
    for course in soup.select('.course-box'):
        title = course.select_one('.course-title').text.strip()
        description = course.select_one('.course-description').text.strip()
        courses.append({'title': title, 'description': description})

    return courses

url = "https://brainlox.com/courses/category/technical"
courses_data = extract_courses(url)
print(courses_data)


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Prepare the documents (title and description) for embedding
documents = [f"{course['title']}: {course['description']}" for course in courses_data]

# Split the text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(documents)

# Store the embeddings in a vector store
vector_store = FAISS.from_documents(docs, embeddings)


In [ ]:
from flask import Flask, request, jsonify
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

app = Flask(__name__)

# Create the conversational retrieval chain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_chain = ConversationalRetrievalChain(vectorstore=vector_store, memory=memory)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({"error": "No message provided"}), 400

    response = conversational_chain({"input": user_input})
    return jsonify({"response": response['output']})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
curl -X POST http://127.0.0.1:5000/chat -H "Content-Type: application/json" -d '{"message": "Tell me about the courses available?"}'